In [2]:
import os
import shutil
import pandas as pd
import csv
import psycopg2
import datetime

In [ ]:
# 1. 파일명에 '사용자정의채널'이 들어간 파일을 읽는다.

# 2. 유효성 검사를 한다.

# 3. 전처리를 한다.

# 4. db에 입력한다.

# 사용자정의채널

In [2]:
# 1. 파일명에 '사용자정의채널'이 들어간 파일을 읽는다.

root_path = 'C:/dataHub'
file_list = os.listdir(root_path)

사용자정의채널 = [file for file in file_list if '사용자정의채널' in file]

사용자정의채널

['사용자정의채널_2023-04-25_2023-04-25.xlsx']

In [3]:
# 2. 유효성 검사를 한다.

checked_file = []
check_date_list = ['2023-04-25']

for check_date in check_date_list:
    for file in 사용자정의채널:
        if check_date + '_' + check_date in file:
            checked_file.append(file)

checked_file

['사용자정의채널_2023-04-25_2023-04-25.xlsx']

In [4]:
# 3. 전처리를 한다.

for file in checked_file:
    df = pd.read_excel(root_path + '/' + file)
    df['날짜'] = file[8:18]
    col_order = df.columns.to_list()[-1:] + df.columns.to_list()[0:-1]
    df = df[col_order]
    df.to_csv(root_path + '/dataSub/사용자정의채널/' + file[:-5] + '.csv', encoding='cp949', index=False)
    os.remove(root_path + '/' + file)

c:\Users\나현중\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [5]:
# 4. db에 입력한다.

conn = psycopg2.connect(host='115.68.228.168', user='postgres', password='cldngks', dbname='phytogether', port=5432)
cursor = conn.cursor()


preprocessed_file = os.listdir(root_path + '/dataSub/사용자정의채널/')
preprocessed_file.remove('DB입력완료')


for file in preprocessed_file:
    df = pd.read_csv(root_path + '/dataSub/사용자정의채널/' + file, encoding='cp949')
    data = df.values.tolist()
    for d in data:
        cursor.execute('INSERT INTO "Naver_Custom_Order" (yymmdd, device, nt_source, nt_medium, nt_detail, nt_keyword, customer_cnt, inflow_cnt, page_cnt, page_inflow_cnt, order_cnt, order_inflow_per, order_price, order_inflow_price, contribute_cnt, contribute_inflow_per, contribute_price, contribute_inflow_price) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)', d)


    shutil.move(root_path + '/dataSub/사용자정의채널/' + file, root_path + '/dataSub/사용자정의채널/DB입력완료')

conn.commit()
conn.close()

# 검색채널

In [6]:
# 1. 파일명에 '검색채널'이 들어간 파일을 읽는다.

root_path = 'C:/dataHub'
file_list = os.listdir(root_path)

검색채널 = [file for file in file_list if '검색채널' in file]

검색채널

['검색채널_2023-04-25_2023-04-25.xlsx']

In [7]:
# 2. 유효성 검사를 한다.

checked_file = []
check_date_list = ['2023-04-25']

for check_date in check_date_list:
    for file in 검색채널:
        if check_date + '_' + check_date in file:
            checked_file.append(file)

checked_file

['검색채널_2023-04-25_2023-04-25.xlsx']

In [8]:
# 3. 전처리를 한다.

for file in checked_file:
    df = pd.read_excel(root_path + '/' + file)
    df['날짜'] = file[5:15]
    df.to_csv(root_path + '/dataSub/검색채널/' + file[:-5] + '.csv', encoding='cp949', index=False)
    os.remove(root_path + '/' + file)

c:\Users\나현중\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [9]:
# 4. db에 입력한다.

conn = psycopg2.connect(host='115.68.228.168', user='postgres', password='cldngks', dbname='phytogether', port=5432)
cursor = conn.cursor()


preprocessed_file = os.listdir(root_path + '/dataSub/검색채널/')
preprocessed_file.remove('DB입력완료')


for file in preprocessed_file:
    df = pd.read_csv(root_path + '/dataSub/검색채널/' + file, encoding='cp949')
    data = df.values.tolist()
    for d in data:
        cursor.execute('INSERT INTO "Naver_Search_Channel" ("A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "yymmdd") VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)', d)


    shutil.move(root_path + '/dataSub/검색채널/' + file, root_path + '/dataSub/검색채널/DB입력완료')

conn.commit()
conn.close()

# 네이버광고_계정1

In [10]:
# 1. 파일명이 '검색광고_운영관리팀,zero2one'으로 시작하는 파일을 읽는다.

root_path = 'C:/dataHub'
file_list = os.listdir(root_path)

네이버광고_계정1 = [file for file in file_list if file.startswith('검색광고_운영관리팀,zero2one')]

네이버광고_계정1

['검색광고_운영관리팀,zero2one (3).csv']

In [11]:
# 2. 유효성 검사를 한다.
# 3. 전처리를 한다.

check_date_list = ['2023-04-25']

if  len(네이버광고_계정1) == 1:

    df = pd.read_csv(root_path + '/' + 네이버광고_계정1[0], skiprows=1, thousands = ',')
    df['날짜'] = df['일별'].apply(lambda x: x.replace('.', '-')[0:-1])
    df['계정'] = 'zero2one'

    

    날짜 = sorted(df['날짜'].unique().tolist())
    
    if 날짜 == sorted(check_date_list):
        df.to_csv(root_path + '/dataSub/네이버광고_계정1/' + 네이버광고_계정1[0], encoding='cp949', index=False)
        os.remove(root_path + '/' + 네이버광고_계정1[0])
        print('유효성 검사 및 전처리 완료')

    else:
        print('실행안됨 : 날짜가 맞지 않습니다.')
else:
    print('실행안됨 : dataHub에 파일이 없습니다.')

유효성 검사 및 전처리 완료


In [12]:
# 4. db에 입력한다.

conn = psycopg2.connect(host='115.68.228.168', user='postgres', password='cldngks', dbname='phytogether', port=5432)
cursor = conn.cursor()


preprocessed_file = os.listdir(root_path + '/dataSub/네이버광고_계정1/')
preprocessed_file.remove('DB입력완료')


if len(preprocessed_file) == 1:
    df = pd.read_csv(root_path + '/dataSub/네이버광고_계정1/' + preprocessed_file[0], encoding='cp949')
    data = df.values.tolist()
    for d in data:
        cursor.execute('INSERT INTO "AD_Naver" ("A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", "AA", "AB", "reg_date", "id") VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)', d)


    shutil.move(root_path + '/dataSub/네이버광고_계정1/' + preprocessed_file[0], root_path + '/dataSub/네이버광고_계정1/DB입력완료')

conn.commit()
conn.close()

# 네이버광고_계정2

In [14]:
# 1. 파일명이 '[zero2one2]검색광고_운영관리팀,zero2one2'으로 시작하는 파일을 읽는다.

root_path = 'C:/dataHub'
file_list = os.listdir(root_path)

네이버광고_계정2 = [file for file in file_list if file.startswith('[zero2one2]검색광고_운영관리팀,zero2one2')]

네이버광고_계정2

['[zero2one2]검색광고_운영관리팀,zero2one2 (2).csv']

'2023-04-26'

In [15]:
# 2. 유효성 검사를 한다.
# 3. 전처리를 한다.

check_date_list = ['2023-04-25']

if  len(네이버광고_계정2) == 1:

    df = pd.read_csv(root_path + '/' + 네이버광고_계정2[0], skiprows=1, thousands = ',')
    df['날짜'] = df['일별'].apply(lambda x: x.replace('.', '-')[0:-1])
    df['계정'] = 'zero2one2'

    날짜 = sorted(df['날짜'].unique().tolist())
    
    if 날짜 == sorted(check_date_list):
        df.to_csv(root_path + '/dataSub/네이버광고_계정2/' + 네이버광고_계정2[0], encoding='cp949', index=False)
        os.remove(root_path + '/' + 네이버광고_계정2[0])
        print('유효성 검사 및 전처리 완료')

    else:
        print('실행안됨 : 날짜가 맞지 않습니다.')
else:
    print('실행안됨 : dataHub에 파일이 없습니다.')

유효성 검사 및 전처리 완료


In [23]:
# 4. db에 입력한다.

conn = psycopg2.connect(host='115.68.228.168', user='postgres', password='cldngks', dbname='phytogether', port=5432)
cursor = conn.cursor()


preprocessed_file = os.listdir(root_path + '/dataSub/네이버광고_계정2/')
preprocessed_file.remove('DB입력완료')


if len(preprocessed_file) == 1:
    df = pd.read_csv(root_path + '/dataSub/네이버광고_계정2/' + preprocessed_file[0], encoding='cp949')
    data = df.values.tolist()
    for d in data:
        cursor.execute('INSERT INTO "AD_Naver" ("A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", "AA", "AB", "reg_date", "id") VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)', d)


    shutil.move(root_path + '/dataSub/네이버광고_계정2/' + preprocessed_file[0], root_path + '/dataSub/네이버광고_계정2/DB입력완료/' + preprocessed_file[0][0:-4] + datetime.date.today().isoformat() + '.csv')

conn.commit()
conn.close()

# 네이버광고_계정3

In [33]:
# 1. 파일명이 '[zero2one3]일별광고보고서,zero2one3'으로 시작하는 파일을 읽는다.

root_path = 'C:/dataHub'
file_list = os.listdir(root_path)

네이버광고_계정3 = [file for file in file_list if file.startswith('[zero2one3]일별광고보고서,zero2one3')]

네이버광고_계정3

['[zero2one3]일별광고보고서,zero2one3 (2).csv']

In [34]:
# 2. 유효성 검사를 한다.
# 3. 전처리를 한다.

check_date_list = ['2023-04-25']

if  len(네이버광고_계정3) == 1:

    df = pd.read_csv(root_path + '/' + 네이버광고_계정3[0], skiprows=1, thousands = ',')
    df['날짜'] = df['일별'].apply(lambda x: x.replace('.', '-')[0:-1])
    df['계정'] = 'zero2one3'

    날짜 = sorted(df['날짜'].unique().tolist())
    
    if 날짜 == sorted(check_date_list):
        df.to_csv(root_path + '/dataSub/네이버광고_계정3/' + 네이버광고_계정3[0], encoding='cp949', index=False)
        os.remove(root_path + '/' + 네이버광고_계정3[0])
        print('유효성 검사 및 전처리 완료')

    else:
        print('실행안됨 : 날짜가 맞지 않습니다.')
else:
    print('실행안됨 : dataHub에 파일이 없습니다.')

유효성 검사 및 전처리 완료


In [35]:
# 4. db에 입력한다.

conn = psycopg2.connect(host='115.68.228.168', user='postgres', password='cldngks', dbname='phytogether', port=5432)
cursor = conn.cursor()


preprocessed_file = os.listdir(root_path + '/dataSub/네이버광고_계정3/')
preprocessed_file.remove('DB입력완료')


if len(preprocessed_file) == 1:
    df = pd.read_csv(root_path + '/dataSub/네이버광고_계정3/' + preprocessed_file[0], encoding='cp949')
    data = df.values.tolist()
    for d in data:
        cursor.execute('INSERT INTO "AD_Naver" ("A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", "AA", "AB", "reg_date", "id") VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)', d)


    shutil.move(root_path + '/dataSub/네이버광고_계정3/' + preprocessed_file[0], root_path + '/dataSub/네이버광고_계정3/DB입력완료/' + preprocessed_file[0][0:-4] + datetime.date.today().isoformat() + '.csv')

else:
    print('파일 : ' + str(preprocessed_file))

conn.commit()
conn.close()

# 쿠팡광고_본계정_상품광고

In [64]:
# 1. 파일명에 'A00197911_pa_daily_keyword'이 들어간 파일을 읽는다.
# 2. 유효성 검사를 한다.

root_path = 'C:/dataHub'
file_list = os.listdir(root_path)

쿠팡광고_본계정_상품광고 = [file for file in file_list if 'A00197911_pa_daily_keyword' in file]

check_date_list = ['2023-04-25']

if (len(쿠팡광고_본계정_상품광고) == 1) and ('A00197911_pa_daily_keyword' + '_' + min(check_date_list).replace('-', '') + '_' + max(check_date_list).replace('-', '') in 쿠팡광고_본계정_상품광고[0]):
    checked_file = 쿠팡광고_본계정_상품광고[0]
    print('유효성 검사 통과')

else:
    print('오류 발생. 확인해보세요')

유효성 검사 통과


In [65]:
# 3. 전처리를 한다.

df = pd.read_excel(root_path + '/' + checked_file)
df['reg_date'] =df['날짜'].astype(str)
df['reg_date'] = df['reg_date'].apply(lambda x: x[0:4] + '-' + x[4:6] + '-' + x[6:8])
df['account'] = 'A00197911'
df = df.fillna('')

numeric_cols = ['총광고수익률(1일)', '직접광고수익률(1일)', '간접광고수익률(1일)', '총광고수익률(14일)', '직접광고수익률(14일)', '간접광고수익률(14일)']

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col].str.rstrip('%'))

df = df.loc[:, [col for col in df.columns if col != '캠페인 ID']]


df.to_csv(root_path + '/dataSub/쿠팡광고_본계정_상품광고/' + checked_file[0:-4] + '.csv', encoding='cp949', index=False)
os.remove(root_path + '/' + checked_file)

c:\Users\나현중\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [66]:
# 4. db에 입력한다.

conn = psycopg2.connect(host='115.68.228.168', user='postgres', password='cldngks', dbname='phytogether', port=5432)
cursor = conn.cursor()


preprocessed_file = os.listdir(root_path + '/dataSub/쿠팡광고_본계정_상품광고/')
preprocessed_file.remove('DB입력완료')


if len(preprocessed_file) == 1:
    df = pd.read_csv(root_path + '/dataSub/쿠팡광고_본계정_상품광고/' + preprocessed_file[0], encoding='cp949', keep_default_na=False)
    data = df.values.tolist()
    for d in data:
        cursor.execute('INSERT INTO "AD_Coupang" ("A", "B", "sales_type", "ad_type", "campaign", "ad_group", "product1", "product1_id", "product2", "product2_id", "adspace", "keyword", "impressions", "clicks", "adcost", "clickrate", "order_cnt", "R", "S", "out_qty", "U", "V", "gross", "X", "Y", "Z", "AA", "AB", "AC", "AD", "AE", "AF", "AG", "AH", "AI", "AJ", "AK", "AL", "AM", "AN", "AO", "AP", "reg_date", "account") VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)', d)
        

    shutil.move(root_path + '/dataSub/쿠팡광고_본계정_상품광고/' + preprocessed_file[0], root_path + '/dataSub/쿠팡광고_본계정_상품광고/DB입력완료/' + preprocessed_file[0])

else:
    print('파일 : ' + str(preprocessed_file))

conn.commit()
conn.close()

In [59]:
df

,날짜,입찰유형,판매방식,광고유형,캠페인명,광고그룹,광고집행 상품명,광고집행 옵션ID,광고전환매출발생 상품명,광고전환매출발생 옵션ID,...,총광고수익률(1일),직접광고수익률(1일),간접광고수익률(1일),총광고수익률(14일),직접광고수익률(14일),간접광고수익률(14일),캠페인 시작일,캠페인 종료일,reg_date,account
0,20230425,cpc,3P,수동 성과형,판토모나 하이퍼포머_수동 성과형 (일반),일반키워드그룹,"판토모나 비오틴 하이퍼포머,240정,3박스",5456714771,"판토모나 비오틴 하이퍼포머,240정,3박스",5456714771,...,0.00,0.00,0.0,0.00,0.00,0.0,2019.11.12,NaN,2023-04-25,A00197911
1,20230425,cpc,3P,수동 성과형,판토모나 하이퍼포머_수동 성과형 (일반),일반키워드그룹,판토모나 비오틴 하이퍼포머 최적 배합 판토텐산 L시스틴 맥주효모 엽산 아연 셀레늄...,5591035825,"판토모나 비오틴 하이퍼포머,240정,3박스",5456714771,...,0.00,0.00,0.0,0.00,0.00,0.0,2019.11.12,NaN,2023-04-25,A00197911
2,20230425,cpc,3P,수동 성과형,판토모나 하이퍼포머_수동 성과형 (일반),일반키워드그룹,판토모나 비오틴 하이퍼포머 최적 배합 판토텐산 L시스틴 맥주효모 엽산 아연 셀레늄...,5591035825,판토모나 비오틴 하이퍼포머 최적 배합 판토텐산 L시스틴 맥주효모 엽산 아연 셀레늄...,5591035825,...,0.00,0.00,0.0,0.00,0.00,0.0,2019.11.12,NaN,2023-04-25,A00197911
3,20230425,cpc,3P,수동 성과형,판토모나 하이퍼포머_수동 성과형 (일반),일반키워드그룹,"판토모나 비오틴 하이퍼포머,240정,3박스",5456714771,"판토모나 비오틴 하이퍼포머,240정,3박스",5456714771,...,0.00,0.00,0.0,0.00,0.00,0.0,2019.11.12,NaN,2023-04-25,A00197911
4,20230425,cpc,3P,수동 성과형,판토모나 하이퍼포머_수동 성과형 (일반),일반키워드그룹,"판토모나 비오틴 하이퍼포머,240정,3박스",5456714771,"판토모나 비오틴 하이퍼포머,240정,3박스",5456714771,...,0.00,0.00,0.0,0.00,0.00,0.0,2019.11.12,NaN,2023-04-25,A00197911
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,20230425,cpc,3P,매출 최적화,트리플마그네슘(3p + 제트)_매출 최적화 2,새 광고 그룹,"파이토웨이 트리플 마그네슘 신경, 근육, 에너지 3중 복합 케어 비타민B6,60정,1개",78206305759,"파이토웨이 트리플 마그네슘 신경, 근육, 에너지 3중 복합 케어 비타민B6,60정,1개",78206305759,...,122.43,122.43,0.0,122.43,122.43,0.0,2023.04.24,NaN,2023-04-25,A00197911
373,20230425,cpc,3P,매출 최적화,트리플마그네슘(3p + 제트)_매출 최적화 2,새 광고 그룹,"파이토웨이 트리플 마그네슘 신경, 근육, 에너지 3중 복합 케어 비타민B6,60정,2개",77396981998,"파이토웨이 트리플 마그네슘 신경, 근육, 에너지 3중 복합 케어 비타민B6,60정,2개",77396981998,...,0.00,0.00,0.0,0.00,0.00,0.0,2023.04.24,NaN,2023-04-25,A00197911
374,20230425,cpc,3P,매출 최적화,트리플마그네슘(3p + 제트)_매출 최적화 2,새 광고 그룹,"파이토웨이 트리플 마그네슘 신경, 근육, 에너지 3중 복합 케어 비타민B6,60정,2개",77396981998,"파이토웨이 트리플 마그네슘 신경, 근육, 에너지 3중 복합 케어 비타민B6,60정,2개",77396981998,...,0.00,0.00,0.0,0.00,0.00,0.0,2023.04.24,NaN,2023-04-25,A00197911
375,20230425,cpc,3P,매출 최적화,트리플마그네슘(3p + 제트)_매출 최적화 2,새 광고 그룹,"파이토웨이 트리플 마그네슘 신경, 근육, 에너지 3중 복합 케어 비타민B6,60정,2개",77396981998,"파이토웨이 트리플 마그네슘 신경, 근육, 에너지 3중 복합 케어 비타민B6,60정,2개",77396981998,...,0.00,0.00,0.0,0.00,0.00,0.0,2023.04.24,NaN,2023-04-25,A00197911
